In [1]:
from ultralytics import YOLO
import numpy as np
from PIL import Image
import os
import csv

In [2]:
# Function to convert relative path to absolute path, ensure the drive letter is lowercase, and normalize the path
def clean_path(path):
    if not os.path.isabs(path):
        path = os.path.abspath(path)
    
    if os.name == 'nt':
        path = path[0].lower() + path[1:]
    
    path = os.path.normpath(path)
    
    parts = path.split(os.path.sep)
    seen = set()
    unique_parts = []
    
    for part in parts:
        if part not in seen:
            unique_parts.append(part)
            seen.add(part)
    
    cleaned_path = os.path.sep.join(unique_parts)
    return cleaned_path

In [3]:
# Load the YOLO model
def load_model():
    relative_path = r"C:\repos\python\Bacteria_counter\BacteriaCounterV1\960px-60_epoch-yolom-augment/best.pt"
    model_path = clean_path(relative_path)
    print(f"Model path resolved to: {model_path}")
    model = YOLO(model_path)
    return model

model = load_model()



Model path resolved to: c:\repos\python\Bacteria_counter\BacteriaCounterV1\960px-60_epoch-yolom-augment\best.pt


In [4]:
# Function to count bacterial colonies in a single image
def count_bacterial_colonies(image_path):
    image = Image.open(image_path)
    image_np = np.array(image)
    
    results = model(image_np, max_det=600, conf=0.50, iou=0.80)
    
    detected_count = sum(len(result.boxes) for result in results)
    
    return detected_count

In [5]:
# Function to count colonies from annotation file
def count_from_annotation(annotation_path):
    if os.path.exists(annotation_path):
        with open(annotation_path, 'r') as file:
            lines = file.readlines()
            return len(lines) 
    return 0

In [6]:
# Function to process all images in a folder and save results to CSV
def process_images_and_save_to_csv(folder_path, csv_file):
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['image name', 'detected colonies', 'expert count (annotation)'])

        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)
            
            annotation_file = os.path.splitext(image_file)[0] + ".txt"
            annotation_path = os.path.join(folder_path, annotation_file)
            
            detected_colonies = count_bacterial_colonies(image_path)
            
            expert_count = count_from_annotation(annotation_path)
        
            writer.writerow([image_file, detected_colonies, expert_count])
            print(f"Processed {image_file}: Detected = {detected_colonies}, Expert Count = {expert_count}")

# Usage
folder_path = r"C:\repos\python\Bacteria_counter\VALIDATION-panel_request\selectedAll-2k"  
csv_output_file = "bacterial_colony_countsAll2k.csv"  # Output CSV file name
process_images_and_save_to_csv(folder_path, csv_output_file)
print(f"Results saved to {csv_output_file}")


0: 960x960 (no detections), 62.0ms
Speed: 12.0ms preprocess, 62.0ms inference, 16.0ms postprocess per image at shape (1, 3, 960, 960)
Processed 10.jpg: Detected = 0, Expert Count = 0

0: 960x896 (no detections), 174.0ms
Speed: 10.0ms preprocess, 174.0ms inference, 6.0ms postprocess per image at shape (1, 3, 960, 896)
Processed 100.jpg: Detected = 0, Expert Count = 0

0: 960x896 4 colonys, 55.0ms
Speed: 11.0ms preprocess, 55.0ms inference, 90.0ms postprocess per image at shape (1, 3, 960, 896)
Processed 1000.jpg: Detected = 4, Expert Count = 4

0: 960x896 87 colonys, 55.0ms
Speed: 10.0ms preprocess, 55.0ms inference, 8.0ms postprocess per image at shape (1, 3, 960, 896)
Processed 1019.jpg: Detected = 87, Expert Count = 95

0: 960x896 (no detections), 55.0ms
Speed: 11.0ms preprocess, 55.0ms inference, 6.0ms postprocess per image at shape (1, 3, 960, 896)
Processed 102.jpg: Detected = 0, Expert Count = 0

0: 960x896 18 colonys, 56.0ms
Speed: 11.0ms preprocess, 56.0ms inference, 10.0ms po